In [1]:
import pandas as pd

from typing import List, Dict
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [2]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[6]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed


In [3]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.run import run_objetivo_1
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.run import run_objetivo_2
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_3.run import run_objetivo_3

In [4]:

# EXTRACT IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.cuadro_averias import (
    extract_averias_table
) 
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.informe_tecnico import (
    extract_tecnico_reports_without_hours_last_dates
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.anexo_indisponibilidad import (
    extract_indisponibilidad_anexos
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.sga_335 import extract_sga_335



In [5]:
# TRANSFORM IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.averias import ( 
    preprocess_df_word_averias
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.informe_tecnico import ( 
    preprocess_df_word_informe_tecnico
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.anexos import ( 
    preprocess_df_word_anexos_indisponibilidad
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_335 import ( 
    preprocess_335
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_380 import ( 
    preprocess_380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.cuismp_sharepoint import ( 
    preprocess_df_cid_cuismp_sharepoint
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.corte_excel import ( 
    preprocess_corte_excel
)


In [6]:
# MERGE IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.excel_sga.excel_sga import ( 
merge_sga_335_corte_excel_sharepoint_cuismp_sga380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.cuadro_averias.datos import ( 
merge_word_datos_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.cuadro_averias.telefonia import ( 
merge_word_telefonia_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.datos import ( 
merge_word_datos_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.telefonia import ( 
merge_word_telefonia_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.anexo_indisponibilidad.datos import ( 
merge_word_datos_anexos_disponibilidad_df_merged_sga
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.anexo_indisponibilidad.telefonia import ( 
merge_word_telefonia_anexos_disponibilidad_df_merged_sga
)


In [7]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 3 3.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_29-04-2025_12-05-2025_20250513_162739.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_29-04-2025_12-05-2025_20250513_163320.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS 9.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS 10.docx"

In [8]:


from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.corte_excel import extract_corte_excel


def all_objetivos(
    path_corte_excel, 
    path_sgq_dinamico_335, 
    path_sga_dinamico_380,
    path_cid_cuismp_sharepoint,
    word_datos_file_path,
    word_telefonia_file_path

) -> List[Dict]:
    """
    Calls each objective's validation function and combines the results.
    Each objective function returns a DataFrame with the columns:
      - 'numero de incidencia'
      - 'mensaje'
      - 'objetivo'
    
    Returns:
      A list of dictionaries (one per incident that fails at least one validation)
      across all objectives.
    """
    results = []
    
    # EXTRACT INVOQUE
    df_corte_excel = pd.read_excel(path_corte_excel, skipfooter=2, engine="openpyxl")

    #df_corte_excel = extract_corte_excel(path_corte_excel, skipfooter=2)

    df_sga_dinamico_335 =  extract_sga_335(path_sgq_dinamico_335)
    df_sga_dinamico_380 = pd.read_excel(path_sga_dinamico_380)
    df_cid_cuismp_sharepoint = pd.read_excel(path_cid_cuismp_sharepoint)

    df_word_datos_averias =  extract_averias_table(word_datos_file_path)
    df_word_telefonia_averias = extract_averias_table(word_telefonia_file_path)
    df_word_datos_informe_tec =  extract_tecnico_reports_without_hours_last_dates(word_datos_file_path)
    df_word_telefonia_informe_tec = extract_tecnico_reports_without_hours_last_dates(word_telefonia_file_path)

    raw_datos_anexos = extract_indisponibilidad_anexos(word_datos_file_path)
    raw_tel_anexos  = extract_indisponibilidad_anexos(word_telefonia_file_path)
    cols = ['ticket', 'indisponibilidad_header', 'indisponibilidad_periodos',
            'indisponibilidad_footer', 'indisponibilidad_total']
    df_word_datos_anexos_indis = (
        raw_datos_anexos
        if raw_datos_anexos is not None
        else pd.DataFrame(columns=cols)
    )
    df_word_telefonia_anexos_indis = (
        raw_tel_anexos
        if raw_tel_anexos is not None
        else pd.DataFrame(columns=cols)
    )

    # TRANSFORM INVOQUE
    df_word_datos_averias = preprocess_df_word_averias(df_word_datos_averias)
    df_word_telefonia_averias = preprocess_df_word_averias(df_word_telefonia_averias)
    
    df_word_datos_informe_tec =  preprocess_df_word_informe_tecnico(df_word_datos_informe_tec)
    df_word_telefonia_informe_tec = preprocess_df_word_informe_tecnico(df_word_telefonia_informe_tec)

    df_word_datos_anexos_indis    = preprocess_df_word_anexos_indisponibilidad(df_word_datos_anexos_indis)
    df_word_telefonia_anexos_indis = preprocess_df_word_anexos_indisponibilidad(df_word_telefonia_anexos_indis)

    df_sga_dinamico_335 = preprocess_335(df_sga_dinamico_335)
    df_sga_dinamico_380 = preprocess_380(df_sga_dinamico_380)
    df_corte_excel = preprocess_corte_excel(df_corte_excel)
    df_cid_cuismp_sharepoint = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
    

    # MERGE INVOQUE

    #  SGA 335 - 380 - SHAREPOINT - CORTE - BOTH
    df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )
    
    #  SGA 335 - 380 - SHAREPOINT - CORTE - LEFT ONLY
    df_unmatched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel,
        df_sga_dinamico_335,
        df_cid_cuismp_sharepoint,
        df_sga_dinamico_380,
        'left_only'
        )
    
    # AVERIAS - DATOS - EXCEL
    df_matched_word_datos_averias_corte_excel = merge_word_datos_averias_corte_excel(
        df_word_datos_averias,
        df_corte_excel,
        'both'
        )
   
    # AVERIAS - TELEFONIA - EXCEL
    df_matched_word_telefonia_averias_corte_excel = merge_word_telefonia_averias_corte_excel(
        df_word_telefonia_averias,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - DATOS - EXCEL
    df_matched_word_datos_informe_tecnico_corte_excel = merge_word_datos_informe_corte_excel(
        df_word_datos_informe_tec,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - TELEFONIA - EXCEL
    df_matched_word_telefonia_informe_tecnico_corte_excel = merge_word_telefonia_informe_corte_excel(
        df_word_telefonia_informe_tec,
        df_corte_excel,
        'both'
        )
    

    df_componente_ii = df_matched_corte_sga335_Sharepoint_cuismp_sga380[
        df_matched_corte_sga335_Sharepoint_cuismp_sga380['COMPONENTE'] == 'COMPONENTE II'
    ]

    #ANEXOS INDISPONIBILIDAD - DATOS - EXCEL
    df_matched_word_datos_anexo_indisponibilidad_df_merged_sga = merge_word_datos_anexos_disponibilidad_df_merged_sga(
        df_componente_ii,
        df_word_datos_anexos_indis,
        )
    

    df_componente_iv = df_matched_corte_sga335_Sharepoint_cuismp_sga380[
        df_matched_corte_sga335_Sharepoint_cuismp_sga380['COMPONENTE'] == 'COMPONENTE IV'
    ]
    
    #ANEXOS INDISPONIBILIDAD - TELEFONIA - EXCEL
    df_matched_word_telefonia_anexo_indisponibilidad_df_merged_sga = merge_word_telefonia_anexos_disponibilidad_df_merged_sga(
        df_componente_iv,
        df_word_telefonia_anexos_indis,
        )
    
    
#  RUN OBJETIVOS 

    obj1_df = run_objetivo_1(
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        df_unmatched_corte_sga335_Sharepoint_cuismp_sga380
        )

    obj2_df = run_objetivo_2(
        df_matched_word_datos_averias_corte_excel,
        df_matched_word_telefonia_averias_corte_excel,
        df_matched_word_datos_informe_tecnico_corte_excel,
        df_matched_word_telefonia_informe_tecnico_corte_excel
        )
    
    obj3_df = run_objetivo_3(
        df_matched_word_datos_anexo_indisponibilidad_df_merged_sga,
        df_matched_word_telefonia_anexo_indisponibilidad_df_merged_sga
    )   

    results.extend(obj1_df.to_dict(orient='records'))
    results.extend(obj2_df.to_dict(orient='records'))
    results.extend(obj3_df.to_dict(orient='records'))

    df_all = pd.DataFrame(results)

    df_grouped = (
        df_all
        .groupby('nro_incidencia', as_index=False)
        .agg({'mensaje': lambda msgs: ' | '.join(msgs),
              'objetivo': lambda objs: ' | '.join(objs), 
              'TIPO REPORTE': 'first', 
              })
    )

    #return df_grouped.to_dict(orient='records')
    return df_grouped

result_df = all_objetivos(
    SAVE_DIR_EXTRACT_EXCEL, 
    SAVE_DIR_EXTRACT_SGA_335, 
    DIR_PARADAS_RELOJ,
    CID_CUISMP_PATH,
    DIR_WORD_DATOS,
    DIR_WORD_TELEFONIA

)

result_df




c:\Users\mozac\Documents\proyectos\hitss\claro\bots_rpa\app\modules\sga\minpub\report_validator\service\objetivos\etl\transform\sga_335.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[dt_cols] = df[dt_cols].apply(pd.to_datetime, errors='coerce', dayfirst=True)
No clock stops found for incident 21812396
No clock stops found for incident 21813871
No clock stops found for incident 21813973
No clock stops found for incident 21814333
No clock stops found for incident 21814420
No clock stops found for incident 21814421
No clock stops found for incident 21814454
No clock stops found for incident 21814456
No clock stops found for incident 21814457
No clock stops found for incident 21814476
No clock stops found for incident 21814510
No clock stops found for incident 21814582
No clock stops found for incident 21814664
No clock stops found for inciden

Converted 4212.0 minutes to 70:12
Converted 195.0 minutes to 03:15
Converted 142.0 minutes to 02:22
Converted 12.0 minutes to 00:12
Converted 3436.0 minutes to 57:16
Converted 309.0 minutes to 05:09
Converted 1.0 minutes to 00:01
Converted 243.0 minutes to 04:03
Converted 589.0 minutes to 09:49
Converted 442.0 minutes to 07:22
Converted 10.0 minutes to 00:10
Converted 299.0 minutes to 04:59
Converted 36.0 minutes to 00:36
Converted 72.0 minutes to 01:12
Converted 67.0 minutes to 01:07
Converted 56.0 minutes to 00:56
Converted 392.0 minutes to 06:32
Converted 28.0 minutes to 00:28
Converted 95.0 minutes to 01:35
Converted 186.0 minutes to 03:06
Converted 183.0 minutes to 03:03
Converted 636.0 minutes to 10:36
Converted 448.0 minutes to 07:28
Converted 426.0 minutes to 07:06
Converted 32.0 minutes to 00:32
Converted 384.0 minutes to 06:24
Converted 293.0 minutes to 04:53
Converted 74.0 minutes to 01:14
Converted 72.0 minutes to 01:12
Converted 232.0 minutes to 03:52
Converted 710.0 minut

,nro_incidencia,mensaje,objetivo,TIPO REPORTE
0,21812186,\n No coincide texto inicio de WORD ANEXOS INDISPONIBILIDAD : \n\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP: \nSe tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s)\n No coincide paradas de reloj de WORD ANEXOS INDISPONIBILIDAD : \n\n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n30/04/2025 09:47:00 hasta el día 01/05/2025 12:30:00\n No coincide total horas sin acceso a la sede de WORD ANEXOS INDISPONIBILIDAD : \n \n es diferente a SGA PAUSA CLIENTE SIN OVERLAP : \n26:43,3.1,RECLAMO
1,21812559,\n La última fecha/hora del parrafo (CUERPO) en columna MEDIDAS CORRECTIVAS - EXCEL: \n05/05/2025 13:00\n no coincide con la columna FECHA Y HORA FIN DE EXCEL - CORTE : \n05/05/2025 13:40,1.8,RECLAMO
2,21813973,"\n EN EXCEL-CORTE PROACTIVO columna MEDIDAS CORRECTIVAS: \nA través de nuestros gestores de monitoreo, Claro identificó la pérdida de conectividad del servicio de datos del cliente, identificado con el CUISMP 11026, y se generó el ticket INC 21813973 el día 03/05/2025 a las 10:32 horas. Inmediatamente, el personal de Claro identificó que hubo un corte de energía comercial no programado. En vista de ello, Claro tomó las acciones correctivas inmediatas, desplazando personal especializado para realizar la colocación de un grupo electrógeno, restableciendo el fluido eléctrico en el punto de presencia a través del sistema de contingencia eléctrico el día 03/05/2025 a las 15:41 horas y verificando el restablecimiento del servicio. \n debe iniciar con 'A través de los Sistemas de Monitoreo de Claro, de manera proactiva se identificó'.",1.7,PROACTIVO
3,21814510,\n CUISMP en Sharepoint CID-CUISMP: \n7004\n es diferente a EXCEL -CORTE (CUISMP): \n7008,1.1,RECLAMO
4,21815127,"\n EN EXCEL-CORTE PROACTIVO columna MEDIDAS CORRECTIVAS: \nA través de los Sistemas de Monitoreo de Claro, de manera proactiva, se identificó la pérdida de gestión del servicio de datos del cliente identificado con el CUISMP 16009 y se generó un ticket el día 05/05/2025 a las 23:04 horas. Inmediatamente, Claro revisó el enlace, encontrando la pérdida de conectividad con los equipos ubicados en la sede del cliente. Posteriormente, se desplazó personal a nuestro punto de presencia. Durante el desplazamiento, el servicio del cliente se restableció por encendido de equipos el día 06/05/2025 a las 06:32 horas, sin la intervención de Claro, comprobándose el correcto funcionamiento y la estabilidad del servicio. \n debe iniciar con 'A través de los Sistemas de Monitoreo de Claro, de manera proactiva se identificó'.",1.7,PROACTIVO
5,21815136,\n (interrupcion_inicio|fecha generacion) en SGA: \n05/05/2025 14:43\n no coincide con FECHA Y HORA INICIO CORTE-EXCEL: \n05/05/2025 14:55 | \n Diferencia en Fecha Fin y Inicio esperado (depende si es masivo) en SGA 335: \n07:06\n no coincide con diferencia en Fecha Inicio y Fin en CORTE-EXCEL (FIN-INICIO (HH:MM)): \n06:54 | \n EL TIEMPO (HH:MM) en CORTE-EXCEL: \n06:54\n no coincide con el tiempo efectivo calculado SGA: \n07:06,1.2 | 1.3 | 1.4,PROACTIVO
6,21815478,\n (interrupcion_inicio|fecha generacion) en SGA: \n06/05/2025 13:17\n no coincide con FECHA Y HORA INICIO CORTE-EXCEL: \n06/05/2025 13:18,1.2,PROACTIVO
7,21815520,"\n EN EXCEL-CORTE PROACTIVO columna MEDIDAS CORRECTIVAS: \nA traves de los gestores de monitoreo de Claro, se detectó la interrupción total del servicio de datos del cliente identificado con el CUISMP 23001 y se genero el ticket, el día 06/05/2025 a las 14:16 horas. Por motivos ajenos a nuestro control, causado por la interrupción en el suministro eléctrico (Corte de energía comercial no programado) en la zona donde se ubica nuestro punto de presencia, afectó a dicho punto de presencia del cual se aprovisiona los servicios del cliente. En vista de ello, CLARO tomó las acciones correctivas inmediatas desplazando personal especializado para realizar los correctivos del caso, restableciendo el fluido eléctrico en el punto d

In [9]:
from datetime import datetime
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent
timestamp = datetime.now().strftime('%Y-%m-%d_%d_%H%M')
SAVE_FINAL_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "final" / f"validacion_total_{timestamp}.xlsx"

result_df.to_excel(SAVE_FINAL_PATH, index=False)
print(f"file saved")

file saved
